# 2-way Single Intersection

In [1]:
import csv
import os
import subprocess

import pyautogui
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from sumo_rl import SumoEnvironment

from helper_functions import evaluate
from reward_functions import diff_wait_time

c:\Users\Justin Mak\anaconda3\envs\rl\lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment GymV26Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [2]:
TEST_NUM = 4

## Create Environment and Instantiate Agent

In [3]:
csv_dir = os.path.join("outputs","2way-single-intersection",f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.mkdir(csv_dir)

env_params = {
    "net_file": os.path.join("nets","2way-single-intersection","single-intersection.net.xml"),
    "route_file": os.path.join("nets","2way-single-intersection","single-intersection-vhvh.rou.xml"),
    "out_csv_name": os.path.join(csv_dir,"ppo"),
    "num_seconds": 1e5,
    "single_agent": True,
    "reward_fn": diff_wait_time,
    "sumo_seed": 42,
}
env = SumoEnvironment(**env_params)

check_env(env)

env = Monitor(env)  # wrap env to know episode reward, length, time

In [4]:
model = PPO(
    "MlpPolicy",
    env,
    learning_rate=5e-3,
    n_steps=256,
    batch_size=256,
    n_epochs=5,
    gamma=0.95,
    gae_lambda=0.99,
    clip_range=0.3,
    ent_coef=0.09,
    vf_coef=0.05,
    max_grad_norm=0.9,
    tensorboard_log=os.path.join("logs","2way-single-intersection"),
    verbose=1
)

Using cuda device
Wrapping the env in a DummyVecEnv.


In [5]:
# Evaluate untrained random agent
mean_reward_untrained, std_reward_untrained = evaluate(
    model, env, n_eval_episodes=1, tb_log_dir=os.path.join("logs","2way-single-intersection",f"PPO_{TEST_NUM}","eval")
)

print(mean_reward_untrained)
print(std_reward_untrained)

-10776595.990128
0.0


## Train and Evaluate Agent

In [6]:
# Train agent
model.learn(total_timesteps=1e5)

Logging to logs\2way-single-intersection\PPO_5
----------------------------
| time/              |     |
|    fps             | 45  |
|    iterations      | 1   |
|    time_elapsed    | 5   |
|    total_timesteps | 256 |
----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 48          |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 512         |
| train/                  |             |
|    approx_kl            | 0.046084046 |
|    clip_fraction        | 0.105       |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.37       |
|    explained_variance   | -0.000114   |
|    learning_rate        | 0.005       |
|    loss                 | 3.45e+04    |
|    n_updates            | 5           |
|    policy_gradient_loss | -0.0203     |
|    value_loss           | 6.91e+05    |
------------------------------------

In [8]:
# Evaluate trained agent
mean_reward_trained, std_reward_trained = evaluate(
    model, env, n_eval_episodes=1, tb_log_dir=os.path.join("logs","2way-single-intersection",f"PPO_{TEST_NUM}","eval_trained")
)

print(mean_reward_trained)
print(std_reward_trained)

-535727.926257
0.0


### Record Results

In [9]:
data = [env_params["net_file"], TEST_NUM, env.reward_fn.__name__, mean_reward_untrained, mean_reward_trained]

with open("test_results.csv", "a", encoding="UTF8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

### Save Model

In [10]:
model.save(os.path.join("saved_models",f"PPO_2way_single_intersection_{TEST_NUM}"))

## Render Video

In [11]:
env_params.update({"use_gui": True})
render_env = SumoEnvironment(**env_params)

check_env(render_env)
render_env.close()

render_env = Monitor(render_env)  # wrap env to know episode reward, length, time

In [12]:
model = PPO.load(os.path.join("saved_models",f"PPO_2way_single_intersection_{TEST_NUM}"))

obs, info = render_env.reset()

folder_path = os.path.join("renders","2way-single-intersection",f"test_{TEST_NUM}")
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

VID_LENGTH = 750
for i in range(VID_LENGTH):
    actions, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = render_env.step(actions)
    env.render()

    im = pyautogui.screenshot(
        os.path.join(folder_path,f"img{i}.jpg"),
        region=(0, 0, 2560, 1542)
    )

render_env.close()  # clean up

In [13]:
subprocess.run([
    "ffmpeg", "-y", "-r", "5", "-i",
    f"renders\\2way-single-intersection\\test_{TEST_NUM}\\img%d.jpg",
    f"videos\\2way_single_intersection_{TEST_NUM}.mp4"
])

CompletedProcess(args=['ffmpeg', '-y', '-r', '5', '-i', 'renders\\2way-single-intersection\\test_4\\img%d.jpg', 'videos\\2way_single_intersection_4.mp4'], returncode=0)

## Clean-up

In [14]:
env.close()
render_env.close()